In [1]:
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch3d
import pytorch3d.io
from pytorch3d.structures import Meshes
from pytorch3d.renderer import TexturesVertex

import trimesh

from src.util import make_faces

edges = torch.tensor([
    [ 0, 22,  2],
    [ 2, 22, 23],#
    [ 3,  7,  2],
    [ 7,  2,  6],#   
    [ 1, 18, 19],
    [ 1, 19,  3],#    
    [ 1, 14, 15],
    [ 1, 14,  0],#    
    [13, 15, 16],
    [15, 16, 18],#    
    [12, 14, 20],
    [14, 22, 20],#    
    [23, 21,  6],
    [ 6,  4, 21],#    
    [ 7, 19, 17],
    [ 5,  7, 17],#    
    [11, 16, 17],
    [11,  9, 16],#
    [ 8,  9, 13],
    [13,  8, 12],#
    [10,  4,  5],
    [11,  5, 10],#    
    [10,  8, 21],
    [20, 21,  8],
])

def pairwise(iterable):
    "s -> (s0, s1), (s2, s3), (s4, s5), ..."
    a = iter(iterable)
    return zip(a, a)

def get_edge_vertices(tri, pair, n):
    square = torch.arange(n**2).reshape(n, n)
    if pair == [0, 1]:
        r = square[0, :]    
    elif pair == [0, 2]:
        r = square[:, 0]
    elif pair == [1, 3]:
        r = square[:, -1]
    elif pair == [2, 3]:
        r = square[-1, :]
    else:
        raise Exception(f'Unknown pair {pair}')
    return ((n ** 2) * tri  + r).tolist()

def get_triangles(l1, l2):
    t1 = [[a, b, c] for a, b, c 
          in zip(l1, l1[1:], l2)]
    t2 = [[a, b, c] for a, b, c 
          in zip(l2, l2[1:], l1[1:])]
    return t1 + t2

In [4]:
def all_edge_triangles(n):
    res, side  = [], 2 ** 2
    for x, y in pairwise(edges.tolist()):
        m = list(set(x + y))
        m.sort()
        a1, a2, b1, b2 = m
        #print(m)        
        pair_a = [a1 % side, a2 % side]
        pair_b = [b1 % side, b2 % side]         
        tri_a, tri_b = a1 // 4, b1 // 4        
        #print( tri_a, b1, tri_b)
        #print(pair_a, pair_b)
        l1 = get_edge_vertices(tri_a, pair_a, n)
        l2 = get_edge_vertices(tri_b, pair_b, n)
        #print(l1, l2)
        res = res + get_triangles(l1, l2)
        #break
    return res
    
res = all_edge_triangles(3)  
res

[[0, 3, 51],
 [3, 6, 52],
 [51, 52, 3],
 [52, 53, 6],
 [6, 7, 15],
 [7, 8, 16],
 [15, 16, 7],
 [16, 17, 8],
 [2, 5, 42],
 [5, 8, 43],
 [42, 43, 5],
 [43, 44, 8],
 [0, 1, 33],
 [1, 2, 34],
 [33, 34, 1],
 [34, 35, 2],
 [29, 32, 36],
 [32, 35, 39],
 [36, 39, 32],
 [39, 42, 35],
 [27, 30, 45],
 [30, 33, 48],
 [45, 48, 30],
 [48, 51, 33],
 [9, 12, 47],
 [12, 15, 50],
 [47, 50, 12],
 [50, 53, 15],
 [11, 14, 38],
 [14, 17, 41],
 [38, 41, 14],
 [41, 44, 17],
 [20, 23, 36],
 [23, 26, 37],
 [36, 37, 23],
 [37, 38, 26],
 [18, 19, 27],
 [19, 20, 28],
 [27, 28, 19],
 [28, 29, 20],
 [9, 10, 24],
 [10, 11, 25],
 [24, 25, 10],
 [25, 26, 11],
 [18, 21, 45],
 [21, 24, 46],
 [45, 46, 21],
 [46, 47, 24]]

In [37]:
n = 2
square = torch.arange(n**2).reshape(n, n)
square

tensor([[0, 1],
        [2, 3]])

In [46]:
square[0, :], square[:, 0], square[:, -1], square[-1, :]

(tensor([0, 1]), tensor([0, 2]), tensor([1, 3]), tensor([2, 3]))

In [49]:
tri = 1 
(get_edge_vertices(tri, [0, 1], n),
 get_edge_vertices(tri, [0, 2], n),  
 get_edge_vertices(tri, [1, 3], n),
 get_edge_vertices(tri, [2, 3], n))

([4, 5], [4, 6], [5, 7], [6, 7])

In [18]:
n = 5
start = -0.5
end = 0.5

d1, d2 = torch.meshgrid(
    torch.linspace(start, end, steps=n), 
    torch.linspace(start, end, steps=n))
d3 = torch.full_like(d1, end) + 1 / n
sides =  OrderedDict({
    'front': torch.stack((+d3,  d1,  d2), dim=-1),
    'right': torch.stack(( d1, +d3,  d2), dim=-1),    
    'back' : torch.stack((-d3,  d1,  d2), dim=-1),         
    'left' : torch.stack(( d1, -d3,  d2), dim=-1),
    'top'  : torch.stack(( d1,  d2, +d3), dim=-1),
    'down' : torch.stack(( d1,  d2, -d3), dim=-1),
})
vert = torch.stack(list(sides.values())).reshape(-1, 3)
offset, faces = n ** 2, make_faces(n, n)
faces = torch.cat([
    i * offset + torch.tensor(faces)
    for i in range(6)])
textures = TexturesVertex(verts_features=[torch.ones_like(vert)])
mesh = Meshes(verts=[vert], faces=[faces], textures=textures)

#n = 3
c0 = 0
c1 = n-1
c2 = n**2-n
c3 = n**2-1 
rmn = torch.tensor(
    [[c0, c2, c2],
     [c3, c3, c3],
     [c0, c1, c1],
     [c1, c3, c1],
     [c1, c2, c3],
     [c2, c3, c2],
     [c0, c0, c0],
     [c1, c2, c0]])

corners = torch.tensor([
    [ 0, 22, 14],
    [ 3, 19,  7],
    
    [16,  9, 13],   
    [17, 11,  5],
    
    [ 1, 18, 15],
    [ 2, 23,  6],
    
    [20,  8, 12],
    [21, 10,  4],
])  
tris = torch.div(corners, n**2, rounding_mode='trunc')
corners = tris * n**2 + rmn

edge_faces = torch.tensor(all_edge_triangles(n))

m = trimesh.Trimesh(vertices=vert, 
    faces=torch.cat((faces, 
                     #corners,
                     edge_faces,
                    )))
m.export(f'./data/cube-{n}x{n}.stl');

In [61]:
edge_faces

[[0, 2, 22],
 [22, 23, 2],
 [2, 3, 6],
 [6, 7, 3],
 [1, 3, 18],
 [18, 19, 3],
 [0, 1, 14],
 [14, 15, 1],
 [13, 15, 16],
 [16, 18, 15],
 [12, 14, 20],
 [20, 22, 14],
 [4, 6, 21],
 [21, 23, 6],
 [5, 7, 17],
 [17, 19, 7],
 [9, 11, 16],
 [16, 17, 11],
 [8, 9, 12],
 [12, 13, 9],
 [4, 5, 10],
 [10, 11, 5],
 [8, 10, 20],
 [20, 21, 10]]

In [16]:
n = 3
start = -0.5
end = 0.5

d1, d2 = torch.meshgrid(
    torch.linspace(start, end, steps=n), 
    torch.linspace(start, end, steps=n))
d3 = torch.full_like(d1, end) + 1 / n
sides =  OrderedDict({
    'front': torch.stack((+d3,  d1,  d2), dim=-1),
    'right': torch.stack(( d1, +d3,  d2), dim=-1),    
    'back' : torch.stack((-d3,  d1,  d2), dim=-1),         
    'left' : torch.stack(( d1, -d3,  d2), dim=-1),
    'top'  : torch.stack(( d1,  d2, +d3), dim=-1),
    'down' : torch.stack(( d1,  d2, -d3), dim=-1),
})
vert = torch.stack(list(sides.values())).reshape(-1, 3)
offset, faces = n ** 2, make_faces(n, n)
faces = torch.cat([
    i * offset + torch.tensor(faces)
    for i in range(6)])
textures = TexturesVertex(verts_features=[torch.ones_like(vert)])
mesh = Meshes(verts=[vert], faces=[faces], textures=textures)

n = 2
c0 = 0
c1 = n-1
c2 = n**2-n
c3 = n**2-1 
rmn = torch.tensor(
[[c0, c2, c2],
 [c3, c3, c3],
 [c0, c1, c1],
 [c1, c3, c1],
 [c1, c2, c3],
 [c2, c3, c2],
 [c0, c0, c0],
 [c1, c2, c0]])
rmn

corners = torch.tensor([
    [ 0, 22, 14],
    [ 3, 19,  7],
    
    [16,  9, 13],   
    [17, 11,  5],
    
    [ 1, 18, 15],
    [ 2, 23,  6],
    
    [20,  8, 12],
    [21, 10,  4],
]) 

adj_corners = tris * n**2 + rmn

m = trimesh.Trimesh(vertices=vert, 
    faces=torch.cat((faces, corners)))
m.export('./data/cube-3x3.stl');

In [15]:
corners - adj_corners

tensor([[ 0, 16, 12],
        [ 0, 12,  4],
        [12,  8, 12],
        [12,  8,  4],
        [ 0, 12, 12],
        [ 0, 16,  4],
        [16,  8, 12],
        [16,  8,  4]])